In [1]:
import os

import fiona
import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio.mask

from rasterio.plot import show
from rasterio.features import shapes
from shapely.geometry import shape
from joblib import delayed, Parallel


In [2]:
data_dir = '/Users/d3y010/projects/atiim/data'

gage_data_file = os.path.join(data_dir, 'tabular', 'water_level.csv')
dem_file = os.path.join(data_dir, 'raster', 'run_1_all.sdat')
basin_shp = os.path.join(data_dir, 'shp', 'basin_1.shp')
gage_shp = os.path.join(data_dir, 'shp', 'gage_location_1.shp')

output_dir = '/Users/d3y010/projects/atiim/test'

run_name = 'test_1'


In [3]:
def process_gage_data(gage_data_file):

    df = pd.read_csv(gage_data_file)

    # convert date and time strings to a pandas datetime type
    df['date_time'] = pd.to_datetime(df['DATE'] + ' ' + df['TIME'], infer_datetime_format=True)

    # sort df by date_time
    df.sort_values(by=['date_time'], inplace=True)

    min_wtr_elev = df['WL_ELEV_M'].min()
    max_wtr_elev = df['WL_ELEV_M'].max()
    wtr_elev_list = df['WL_ELEV_M'].tolist()
    day_part, hour_interval = 1, 1
    d_freq = df['WL_ELEV_M'].value_counts().to_dict()
    
    return min_wtr_elev, max_wtr_elev



def create_basin_dem(basin_shp, dem_file, output_directory, run_name):
    """Mask the input DEM using a basin geometry representative of the contributing area.
    
    
    """

    # dissolve target basin geometries
    basin_geom = gpd.read_file(basin_shp).dissolve().geometry.values[0]
    
    with rasterio.open(dem_file) as src:
        
        # apply basin geometry as a mask
        out_image, out_transform = rasterio.mask.mask(src, basin_geom, crop=True)
        
        # update the raster metadata with newly cropped extent
        out_meta = src.meta
        out_meta.update({"driver": "GTiff",
                         "height": out_image.shape[1],
                         "width": out_image.shape[2],
                         "transform": out_transform})

        # write outputs
        output_file = os.path.join(output_directory, f"dem_masked_{run_name}.tif")
        with rasterio.open(output_file, "w", **out_meta) as dest:
            dest.write(out_image)
            
        return output_file


def process_slice(arr, upper_elev, transform, output_directory, target_crs):
    """Create a water level polygon shapefile containing a single feature that represents
    the grid cells of an input DEM that are less than or equal to an upper elevation level.
    
    """

    # create every value greater than or equal to the upper elevation to 1, others to 0
    arx = np.where(arr <= upper_elev, 1, 0).astype(np.int16)

    # build each feature based on the extracted grid cells from the array
    results = list(
        {'properties': {'raster_val': val}, 'geometry': shp}
        for index, (shp, val)  in enumerate(
            shapes(arx, mask=None, transform=transform))
    )

    # list of geometries
    geoms = list(results)

    # build geopandas dataframe from geometries
    gdf = gpd.GeoDataFrame.from_features(geoms, crs=target_crs)

    # only keep the ones
    gdf = gdf.loc[gdf['raster_val'] == 1]

    # dissolve into a single polygon
    gdf = gdf.dissolve('raster_val')

    # write to file
    out_file = os.path.join(output_directory, f'wl_{int(upper_elev)*10}.shp')
    gdf.to_file(out_file)


def generate_water_polygons(dem_file, 
                            basin_shp, 
                            gage_data_file, 
                            output_directory, 
                            run_name, 
                            elevation_interval=0.1):

    # process gage data file
    min_gage_elev, max_gage_elev = process_gage_data(gage_data_file)
    
    with rasterio.Env():
        
        # clip the input DEM to a target basin contributing area
        masked_dem_file = create_basin_dem(basin_shp, dem_file, output_directory, run_name)

        with rasterio.open(masked_dem_file) as src:

            # read the raster band into a number array
            arr = src.read(1)

            # convert the raster nodata value to numpy nan
            arr[arr == src.nodata] = np.nan

            raster_min = np.nanmin(arr)
            raster_max = np.nanmax(arr)
            
            # use the minimum bounding elevation e.g., the max of min available
            elev_min = max([min_gage_elev, raster_min])
            
            # use the maximum bounding elevation e.g., the min of max available
            elev_max = min([max_gage_elev, raster_max])
            
            # construct elevation upper bounds to process for each slice
            elev_slices = np.arange(elev_min, elev_max, elevation_interval)
            
            # replace the last slice upper bound with the max elevation
            elev_slices[-1] = elev_max

            # process all elevation slices in parallel
            Parallel(n_jobs=-1)(
                delayed(process_slice)(arr, i, src.transform, output_directory, src.crs) 
                for i in elev_slices)


In [4]:
%%time

generate_water_polygons(dem_file, 
                        basin_shp, 
                        gage_data_file, 
                        output_dir, 
                        run_name, 
                        elevation_interval=0.1)


DriverIOError: .shp file is unreadable, or corrupt.